In [51]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [52]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [53]:
%pip install playwright

Note: you may need to restart the kernel to use updated packages.


In [54]:
!playwright install

In [55]:
SEASONS = list(range(2016, 2024))          

In [56]:
SEASONS

[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [57]:
DATA_DIR = "data"
SCHEDULE_DIR = os.path.join(DATA_DIR, "schedule")
SCORES_DIR = os.path.join(DATA_DIR, "scores")
#Joins the other folders with the data

In [58]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep*i) #Pauses program so you don't get banned

        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch() #Opens open source chrome window
                page = await browser.new_page() #Creating a new tab
                await page.goto(url) #Send the browser in the tab to a certain page
                print(await page.title())
                html = await page.inner_html(selector) #Only takes specific html
        except PlaywrightTimeout:
                print(f"Timeout error on {url}")
                continue
        else: 
                break
    return html

In [59]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]

    for url in standings_pages:
        save_path = os.path.join(SCHEDULE_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [60]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com


In [62]:
schedule_files = os.listdir(SCHEDULE_DIR)

In [69]:
async def scrape_game(schedule_file):
    with open(schedule_file, 'r') as f:
        html = f.read()
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]
    
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
    
        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [70]:
schedule_files = [s for s in schedule_files if ".html" in s]

In [ ]:
for f in schedule_files:
    filepath = os.path.join(SCHEDULE_DIR, f)
    await scrape_game(filepath)

Timeout error on https://www.basketball-reference.com/boxscores/202106010BRK.html
Timeout error on https://www.basketball-reference.com/boxscores/202106010BRK.html
Timeout error on https://www.basketball-reference.com/boxscores/202106010BRK.html
Timeout error on https://www.basketball-reference.com/boxscores/202106010DEN.html
Timeout error on https://www.basketball-reference.com/boxscores/202106010DEN.html
2021 NBA Western Conference First Round Game 5: Trail Blazers vs Nuggets, June 1, 2021 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202106010PHO.html
2021 NBA Western Conference First Round Game 5: Lakers vs Suns, June 1, 2021 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202106020PHI.html
Timeout error on https://www.basketball-reference.com/boxscores/202106020PHI.html
Timeout error on https://www.basketball-reference.com/boxscores/202106020PHI.html
Timeout error on https://www.basketball-refe